In [1]:
import pandas as pd
import re
from datetime import datetime
from time import sleep
from selenium.common.exceptions import TimeoutException, NoSuchElementException
# Criar uma instância do navegador
from selenium import webdriver
# Definir o caminho do chromedriver.exe
from selenium.webdriver.chrome.service import Service
# Fornece a classe By, usada para especificar como localizar elementos na página
from selenium.webdriver.common.by import By
# Permite criar um esperador explícito, que aguarda um tempo até que uma condição seja atendida
from selenium.webdriver.support.ui import WebDriverWait
# Importa um conjunto de condições pré-definidas que podem ser usadas com WebDriverWait, como
from selenium.webdriver.support import expected_conditions as EC

In [2]:
def coleta_pag(n_pag):
    if n_pag == 1:
        sleep(1)
        return 'https://www.pichau.com.br/hardware/placa-de-video?placadevideo=129,130&page=1'
    else:
        return f'https://www.pichau.com.br/hardware/placa-de-video?placadevideo=129,130&page={n_pag}'

In [3]:

def coleta_dados(url):
    service = Service()
    options = webdriver.ChromeOptions()
    options.add_argument("--disable-blink-features=AutomationControlled") # Desabilita o modo de automação do Chrome
    options.add_argument("start-maximized") # Inicia o navegador maximizado
    options.add_experimental_option("excludeSwitches", ["enable-automation"]) # Exclui o switch de automação
    options.add_experimental_option('useAutomationExtension', False) # Desabilita a extensão de automação do Chrome
    driver = webdriver.Chrome(service=service, options=options)
    driver.get(url)

    title = driver.find_elements(By.TAG_NAME, 'h2')[0:36]
    tamanho = len(title)
    lista = []
    tipomemoria = None
    
    for placas in range(tamanho):
        try:
            
            loc = WebDriverWait(driver, 12).until(
                EC.presence_of_all_elements_located((By.TAG_NAME, 'h2')))
            try:
                esgotado = WebDriverWait(driver, 2).until(EC.presence_of_element_located(
                    (By.CLASS_NAME, 'mui-8rpawh-out_of_stock'))).text
                if esgotado:
                    continue
            except:
                pass

            
            loc[placas].click()
            sleep(1.5)
            WebDriverWait(driver, 5).until(EC.presence_of_element_located(
                    (By.TAG_NAME, 'h1')))

            try:
                modelo = WebDriverWait(driver, 12).until(EC.presence_of_element_located(
                    (By.TAG_NAME, 'h1'))).text.split(",")[0].strip()
                precoavista = WebDriverWait(driver, 12).until(
                    EC.presence_of_element_located((By.CLASS_NAME, 'mui-1q2ojdg-price_vista'))).text
                precoparcelado = WebDriverWait(driver, 12).until(
                    EC.presence_of_element_located((By.CLASS_NAME, 'mui-7ie9un-price_total'))).text

            except NoSuchElementException as e:
                print(f"Erro ao coletar dados da placa {placas}: {e}")
                modelo = None
                precoavista = None
                precoparcelado = None
                
            driver.execute_script(
                "window.scrollTo(0, document.body.scrollHeight);")

            try:
                itens = WebDriverWait(driver, 15).until(EC.presence_of_all_elements_located(
                    (By.CSS_SELECTOR, 'table.table-specification tr')))
            except TimeoutException:
                print(f"Erro ao localizar especificações da placa {placas}")
                itens = []

            dados = {
                "Data": None,
                "Modelo": modelo,
                "Vram": None,
                "Tipo de Memoria": None,
                "Preco a vista": precoavista,
                "Preco Parcelado": precoparcelado
            }

            for item in itens:
        
                texto_th = item.find_element(
                    By.TAG_NAME, 'th').text.strip().upper()
                texto_td = item.find_element(
                    By.TAG_NAME, 'td').text.strip().upper()

                if texto_th.startswith(("MEMÓRIA:", "MEMÓRIA DE VÍDEO:", "TAMANHO DA MEMÓRIA:", "TAMANHO DE MEMÓRIA:", "CAPACIDADE DA MEMÓRIA:", "TAMANHO E TIPO DE MEMÓRIA:")):
                    loc_vram = re.search(r'\d+\s*GB', texto_td)
                    vram = loc_vram.group().replace(" ", "") if loc_vram else None
                    loc_tipo = re.search(r'GDDR\d+[X]?', texto_td)
                    tipo = loc_tipo.group().replace(" ", "") if loc_tipo else None

                    if tipo:
                        tipomemoria = tipo
                    if vram:
                        dados["Vram"] = vram

                elif texto_th.startswith(("TIPO DE MEMÓRIA:", "TIPO DRAM:")):
                    dados["Tipo de Memoria"] = texto_td

            if not dados["Tipo de Memoria"] and tipomemoria:
                dados["Tipo de Memoria"] = tipomemoria

            lista.append(dados)
            sleep(1)
            driver.back()
        except Exception as e:
            print(f"Erro ao processar a placa{placas}: {e}")  
            driver.back()  
    driver.quit()
    return lista

In [4]:
numero_pagina = 5
dataframes = []
for pag in range(1,numero_pagina + 1):
    url = coleta_pag(pag)
    dfsite = coleta_dados(url)
    dataframes.extend(dfsite)
    
data_atual = datetime.now().strftime('%d-%m-%y') 
nome_arquivo = f'Placas_{data_atual}.csv'

In [5]:
df = pd.DataFrame(dataframes)
df["Data"] = data_atual
df["Modelo"] = df["Modelo"].str.extract(r'PLACA DE VIDEO (.*)', expand= False)
df.to_csv(nome_arquivo, index= False)